# Shark Attack Project

### Created by Alvaro, Aurelie, Clara and Marc

For our project, we assessed and processed the data based on our business proposal of a "shark repellent" (à la snake oil). We are looking mostly at who we should advertise to and in which countries those advertisements should run, thus only cleaning and processing for the columns that concern our marketing/ads strategy.

In [ ]:
import pandas as pd
import numpy as np
import re #for handling the date column

#for data visualization
import matplotlib.pyplot as plt
import seaborn as sns

url = 'https://www.sharkattackfile.net/spreadsheets/GSAF5.xls'
shark_df = pd.read_excel(url)

# Working with a copy in order not to overwrite the original dataset with code with errors
shark_df_copy = shark_df.copy()
shark_df_copy

In [ ]:
shark_df_copy.info()
#We can see that some columns are hidden in the Excel file and are unnecessary for our research, therefore we should drop them.

In [ ]:
#Drop columns from 15 to 22
columns_to_drop = shark_df_copy.columns[15:23]
shark_df_copy = shark_df_copy.drop(columns=columns_to_drop, axis=1)

#Standardizing the names of every column 
shark_df_copy.columns = shark_df_copy.columns.str.strip().str.lower().str.replace(' ', '_')

#Dropping every row that has the value 'Invalid' in its type to prevent working with useless data
shark_df_copy.drop(shark_df_copy[shark_df_copy['type'] == 'Invalid'].index, inplace = True)
shark_df_copy

In [ ]:
#Dropping all of the unnecessary columns for our research
shark_df_copy = shark_df_copy.drop(columns=['state', 'location', 'name', 'source', 'year'])

In [ ]:
def clean_season(shark_df_copy):
    #date format
    shark_df_copy['date'] = pd.to_datetime(shark_df_copy['date'], errors='coerce').dt.date
    shark_df_copy.dropna(subset=['date'], inplace=True) # drop a few use case I can't format
    shark_df_copy.drop_duplicates(subset=['date'], inplace=True)
    shark_df_copy.sort_values(by='date', inplace=True) # not sure if we keep this line it just sort by date
    
    
    # function to differentiate the hemispheres and convert dates to seasons
    def determine_hemisphere(country):
        southern_hemisphere = ['australia', 'brazil', 'south africa', 'argentina', "seychelles"]  # to determine later
        return 'south' if country in southern_hemisphere else 'north'
    #create a hemisphere column just for test purpose
    shark_df_copy['hemisphere'] = shark_df_copy['country'].apply(determine_hemisphere)
    
    def get_season(date, hemisphere):
        month = date.month
        if hemisphere == 'north':
            if month in [12, 1, 2]:
                return 'Winter'
            elif month in [3, 4, 5]:
                return 'Spring'
            elif month in [6, 7, 8]:
                return 'Summer'
            else:
                return 'Fall'
        else:  # South
            if month in [12, 1, 2]:
                return 'Summer'
            elif month in [3, 4, 5]:
                return 'Fall'
            elif month in [6, 7, 8]:
                return 'Winter'
            else:
                return 'Spring'
    
    shark_df_copy['season'] = shark_df_copy.apply(lambda row: get_season(row['date'], row['hemisphere']), axis=1)
    display(shark_df_copy)
clean_season(shark_df_copy)

In [ ]:
#function for times values and part of the day
def clean_time_and_time_of_day(shark_df_copy):
    def clean_time(time_str):
        if pd.isna(time_str):
            return None #fill with mean() 
        # Remove non-numeric and non-h characters
        time_str = re.sub(r'[^0-9h]', '', time_str)
    
        # If the time format is hMM or hhMM
        match = re.match(r'(\d{1,2})h(\d{2})$', time_str)
        if match:
            return f"{match.group(1).zfill(2)}:{match.group(2)}"
    
        # If the time format is hhh or hhMM
        match = re.match(r'(\d{1,3})h$', time_str)
        if match:
            return pd.to_datetime(match.group(1).zfill(4), format='%H%M', errors='coerce').strftime('%H:%M')
    
        # Default case to handle any other remaining formats
        match = re.match(r'(\d{1,2})h(\d{2})?$', time_str)
        if match:
            hour = match.group(1).zfill(2)
            minute = match.group(2) if match.group(2) else '00'
            return f"{hour}:{minute}"
        return None #fill with mean() 
    

    
    def time_of_day(hour):
        if 5 <= hour < 12:
            return 'Morning'
        elif 12 <= hour < 17:
            return 'Afternoon'
        elif 17 <= hour < 21:
            return 'Evening'
        else:
            return 'Night'
    
    
    # Ensure 'time' column is properly formatted as string
    shark_df_copy['time'] = shark_df_copy['time'].astype(str)
    shark_df_copy['time'] = shark_df_copy['time'].apply(clean_time)
    
    valid_times = pd.to_datetime(shark_df_copy['time'], format='%H:%M', errors='coerce')
    
    mean_time = valid_times.dropna().mean()
    mean_time_str = mean_time.strftime('%H:%M')
    shark_df_copy['time'] = shark_df_copy['time'].fillna(mean_time_str)
    
    shark_df_copy['time'] = pd.to_datetime(shark_df_copy['time'], format='%H:%M', errors='coerce').dt.time
    
    shark_df_copy['time_of_day'] = shark_df_copy['time'].apply(lambda x: time_of_day(x.hour) if pd.notnull(x) else 'Invalid Time')
    display(shark_df_copy)

clean_time_and_time_of_day(shark_df_copy)

In [ ]:
#Cleaning the sex column and replacing the typos for the intended gender
def clean_sex(shark_df_copy):
  shark_df_copy["sex"] = shark_df_copy["sex"].str.strip()
  shark_df_copy["sex"] = shark_df_copy["sex"].replace("M x 2", "M")
  shark_df_copy["sex"] = shark_df_copy["sex"].replace("N", "M")
  shark_df_copy["sex"] = shark_df_copy["sex"].replace("lli", "M")
  shark_df_copy["sex"] = shark_df_copy["sex"].replace(".", "M")

  value_counts = shark_df_copy['sex'].value_counts(dropna=False)
  display(value_counts)

clean_sex(shark_df_copy)

In [ ]:
#Cleaning the country column and dropping all rows with null values in it.
def clean_country(shark_df_copy):
    shark_df_copy.dropna(subset=["country"], inplace=True) 
    shark_df_copy["country"] = shark_df_copy['country'].str.lower().str.replace(' ', '_') 
    primary_countries = shark_df_copy["country"].value_counts() #show the primary countries where shark attacks occur
    total_attacks = shark_df_copy['country'].count()
    percentage_by_country = (primary_countries/total_attacks)*100

    display(primary_countries.head(20))
    print('\n')
    display(percentage_by_country)
clean_country(shark_df_copy)

In [ ]:
#Cleaning the activity column by creating a priority map allows us to change the initial values to "categories" and make the data more understandable.
#In this map, order matters and we use it to our advantage.
def clean_activity(shark_df_copy):
  priority_map = [
      (['disaster', 'adrift', 'sunk', 'sink', 'wreck', 'founde', 'sank', 'capsiz', 'overboard'], 'Disaster'), #this is the first priority because we do not want to overwrite "overboard" as "Surfing/ Boarding"
      (['surf', 'board', 'sruf', 'paddl', 'sup'], 'Surfing/ Boarding'),
      (['swim', 'float', 'swm'], 'Swimming/ Floating'),
      (['fish', 'lobster', 'scallop', 'shrimp', 'crab', 'oyster', 'hunt', 'harpoon', 'fihi'], 'Fishing'),
      (['boat', 'kayak', 'ship', 'raft', 'canoe', 'row', 'sail', 'yacht', 'jet ski'], 'Boating/ Watercraft'),
      (['wad'], 'Wading'),
      (['bath'], 'Bathing'),
      (['diving', 'dive'], 'Diving'),
      (['snork'], 'Snorkeling'),
      (['stand', 'sit', 'squat'], 'Standing/ Sitting'),
      (['shark'], 'Handling/ Looking at Shark'),
      (['play'], 'Playing')
  ]

#
  def categorize_activity_priority(activity):
      if pd.isna(activity): #for missing values
          return "Unknown"
      for keywords, category in priority_map:
          if any(keyword in activity.lower() for keyword in keywords):
              return category
      return "Other"

#looking at the different categories and how many values of each there are to compare with the uncleaned version
  shark_df_copy['activity'] = shark_df_copy['activity'].apply(categorize_activity_priority)
  print(shark_df_copy['activity'].value_counts())
  return shark_df_copy

clean_activity(shark_df_copy)

In [ ]:
# Cleaning the unnamed col, renaming it and change any values that might be typos
def clean_fatal(shark_df_copy):
    # Get the column name at index 7
    col_name = shark_df_copy.columns[7]
    
    # Rename the column
    shark_df_copy.rename(columns={col_name: 'was_it_fatal'}, inplace=True)
    
    # Ensure the column was renamed correctly
    if 'was_it_fatal' in shark_df_copy.columns:
        shark_df_copy['was_it_fatal'] = shark_df_copy['was_it_fatal'].str.lower().str.strip()

    # Checking the current values
    print(shark_df_copy['was_it_fatal'].value_counts())
    print(shark_df_copy['was_it_fatal'].isna().sum()) 

    # Creating a map that changes the typos for their intended value
    fatality_map = {
    'y': 'Yes',
    'f': 'Yes',
    'y x 2': 'Yes',
    'n': 'No',
    'm': 'No',
    'nq': 'No',
    'unknown': 'Unknown',
    }

    print('\n')
    shark_df_copy['was_it_fatal'] = shark_df_copy['was_it_fatal'].map(fatality_map).fillna('Unknown')
    print(shark_df_copy['was_it_fatal'].value_counts()) 
clean_fatal(shark_df_copy)

In [ ]:
#Cleaning and categorizing the ages
def clean_age(shark_df_copy):
    # Calculating the frequency of each age and filter for values with freq > 1
    frequency = shark_df_copy["age"].value_counts()
    values_to_keep = frequency[frequency > 1].index
    
    # Dropping every null value and values with frequency = 1
    shark_df_copy = shark_df_copy[shark_df_copy['age'].isin(values_to_keep)]
    
    # Replacing "N/A" and non-numeric values with NaN
    shark_df_copy.loc[:, 'age'] = pd.to_numeric(shark_df_copy['age'], errors='coerce')
    shark_df_copy = shark_df_copy.dropna(subset=['age'])
    
    # Converting age to integers now that we only have numeric values
    shark_df_copy.loc[:, 'age'] = shark_df_copy['age'].astype(int)
    
    # Defining age bins and labels
    bins = [0, 12, 20, 30, 45, 64, float('inf')]
    labels = ['Children', 'Teenagers', 'Young Adults', 'Middle-aged Adults', 'Older Adults', 'Seniors']
    
    # Applying pd.cut to categorize ages
    shark_df_copy.loc[:, 'age_category'] = pd.cut(shark_df_copy['age'], bins=bins, labels=labels, right=True)

    return shark_df_copy

# Updating the dataframe to have the new column
shark_df_copy = clean_age(shark_df_copy)

clean_age(shark_df_copy)

In [ ]:
# Calculating frequencies and percentages of attacks by age category
freq = shark_df_copy["age_category"].value_counts()
total_attacks = freq.sum()
percentage = freq / total_attacks * 100

# Creating a DataFrame with the Frequency and Percentage columns
summary_shark_df_copy = pd.DataFrame({'Frequency': freq, 'Percentage': percentage})

display(summary_shark_df_copy)

In [ ]:
# Pivot table for attacks per age category and sex
pivot_table = shark_df_copy.pivot_table(index='age_category', columns='sex', aggfunc='size', fill_value=0)
print(pivot_table)

In [ ]:
#Activities visualization
plt.figure(figsize=(12, 6))
activity_counts = shark_df_copy['activity'].value_counts()
sns.barplot(x=activity_counts.index, y=activity_counts.values , palette='hls')
plt.title('Shark Attacks by Activity Type', fontsize=15)
plt.xlabel('Activity', fontsize=12)
plt.ylabel('Number of Attacks', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# Fatalities pie
plt.figure(figsize=(10, 7))
fatal_counts = shark_df_copy['was_it_fatal'].value_counts()
plt.pie(fatal_counts, labels=fatal_counts.index, autopct='%1.1f%%',
        colors=['indianred', 'orange', 'lightgray'])
plt.title('Shark Attack Fatality Rates', fontsize=12)
plt.axis('equal')
plt.show()

In [ ]:
#Attacks by time of the day

plt.figure(figsize=(10, 6))
time_counts = shark_df_copy['time_of_day'].value_counts()
sns.barplot(x=time_counts.index, y=time_counts.values,
            order=['Morning', 'Afternoon', 'Evening', 'Night'], palette='rocket')
plt.title('Shark Attacks by Time of Day', fontsize=15)
plt.xlabel('Time of Day', fontsize=12)
plt.ylabel('Number of Attacks', fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
#Shark attacks by age category

plt.figure(figsize=(12, 6))
age_counts = shark_df_copy['age_category'].value_counts()
sns.barplot(x=age_counts.index, y=age_counts.values, palette='rocket')
plt.title('Shark Attacks by Age Category', fontsize=15)
plt.xlabel('Age Category', fontsize=12)
plt.ylabel('Number of Attacks', fontsize=12)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
#Shark attacks by season

plt.figure(figsize=(10, 6))
time_counts = shark_df_copy['season'].value_counts()
sns.barplot(x=time_counts.index, y=time_counts.values,
            order=['Summer', 'Fall', 'Winter', 'Spring'], palette='husl')
plt.title('Shark Attacks by Season', fontsize=15)
plt.xlabel('Time of Day', fontsize=12)
plt.ylabel('Number of Attacks', fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
#Shark attacks by country

countries_counts = shark_df_copy['country'].value_counts().head(10)

plt.figure(figsize=(12, 6))
sns.barplot(x=countries_counts.index, y=countries_counts.values, palette='husl')
plt.title('Shark Attacks by Country', fontsize=15)
plt.xlabel('Countries', fontsize=12)
plt.ylabel('Number of Attacks', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

In [ ]:
#Shark attacks by gender

gender_counts = shark_df_copy['sex'].value_counts()

plt.figure(figsize=(12, 6))
sns.barplot(x=gender_counts.index, y=gender_counts.values, palette='husl')
plt.title('Shark Attacks by Gender', fontsize=15)
plt.xlabel('Gender', fontsize=12)
plt.ylabel('Number of Attacks', fontsize=12)
#plt.xticks(rotation=45, ha='right')
plt.tight_layout()